In [1]:
from dotenv import load_dotenv
from google.adk.agents import Agent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.tools import google_search
from google.genai import types
from pydantic import BaseModel, Field


In [2]:
load_dotenv()

True

In [3]:
class FootballClubList(BaseModel):
    clubs: list[str] = Field(
        description="A list of exactly 10 major European football club names."
    )

In [4]:
retry_config=types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1, # Initial delay before first retry (in seconds)
    http_status_codes=[429, 500, 503, 504] # Retry on these HTTP errors
)

In [5]:
agent_lister = Agent(
    name="lister",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config,
    ),
    instruction="List 10 major European football clubs.",
    output_schema=FootballClubList,
)

In [6]:
runner_lister = InMemoryRunner(agent=agent_lister)
response_lister_raw = await runner_lister.run_debug(
    "List 10 major European football clubs"
)



 ### Created new session: debug_session_id

User > List 10 major European football clubs
lister > {
  "clubs": [
    "Real Madrid",
    "Barcelona",
    "Bayern Munich",
    "Manchester United",
    "Liverpool",
    "Juventus",
    "Paris Saint-Germain",
    "AC Milan",
    "Inter Milan",
    "Borussia Dortmund"
  ]
}


In [7]:
response_lister = FootballClubList.model_validate_json(response_lister_raw[0].content.parts[0].text)
clubs =response_lister.clubs
print("clubs:", clubs)


clubs: ['Real Madrid', 'Barcelona', 'Bayern Munich', 'Manchester United', 'Liverpool', 'Juventus', 'Paris Saint-Germain', 'AC Milan', 'Inter Milan', 'Borussia Dortmund']


In [8]:
print("Select a club:")
for i, club in enumerate(clubs, 1):
    print(f"{i}. {club}")

selection = int(input("Enter number: "))
selected_club = clubs[selection - 1]
print(f"You selected: {selected_club}")

Select a club:
1. Real Madrid
2. Barcelona
3. Bayern Munich
4. Manchester United
5. Liverpool
6. Juventus
7. Paris Saint-Germain
8. AC Milan
9. Inter Milan
10. Borussia Dortmund
You selected: Manchester United
